In [6]:
# the code we have in the previous module,
import pygame
pygame.init()

win = pygame.display.set_mode((500,480))
pygame.display.set_caption("FIRST GAME")
screen_width = 480
bg = pygame.image.load("resources/bg.jpg")
clock = pygame.time.Clock()


class Player():
    walkRight = [pygame.image.load("resources/R1.png"), pygame.image.load("resources/R2.png"), pygame.image.load("resources/R3.png"), pygame.image.load("resources/R4.png"), pygame.image.load("resources/R5.png"), pygame.image.load("resources/R6.png"), pygame.image.load("resources/R7.png"), pygame.image.load("resources/R8.png"), pygame.image.load("resources/R9.png")] 
    walkLeft = [pygame.image.load("resources/L1.png"), pygame.image.load("resources/L2.png"), pygame.image.load("resources/L3.png"), pygame.image.load("resources/L4.png"), pygame.image.load("resources/L5.png"), pygame.image.load("resources/L6.png"), pygame.image.load("resources/L7.png"), pygame.image.load("resources/L8.png"), pygame.image.load("resources/L9.png")]

    def __init__(self,x,y,h,w):
        self.x = x
        self.y = y
        self.h = h
        self.w = w
        self.vel = 10
        self.isJump = False
        self.jumpCount = 10
        self.left = False
        self.right = False
        self.walkCount = 0
        self.standing = True
    

    def draw(self,win):
        if self.walkCount+1 >= 27:
            self.walkCount = 0
        
        if not self.standing:
            if self.left:
                win.blit(self.walkLeft[self.walkCount//3], (self.x,self.y))
                self.walkCount += 1
            elif self.right:
                win.blit(self.walkRight[self.walkCount//3], (self.x,self.y))
                self.walkCount += 1
        else:
            if self.right:
                win.blit(self.walkRight[0], (self.x,self.y))
            else:
                win.blit(self.walkLeft[0], (self.x,self.y))
            

class Projectile():
    def __init__(self,x,y,radius,color,facing):
        self.x = x
        self.y = y
        self.radius = radius
        self.color = color
        self.facing = facing
        self.vel = 15 * facing
        
    def draw(self,win):
        pygame.draw.circle(win,self.color,(self.x,self.y),self.radius)


class Enemy:
    walkRight = [pygame.image.load("resources/R1E.png"), pygame.image.load("resources/R2E.png"), pygame.image.load("resources/R3E.png"), pygame.image.load("resources/R4E.png"), pygame.image.load("resources/R5E.png"), pygame.image.load("resources/R6E.png"), pygame.image.load("resources/R7E.png"), pygame.image.load("resources/R8E.png"), pygame.image.load("resources/R9E.png"), pygame.image.load("resources/R10E.png"), pygame.image.load("resources/R11E.png")] 
    walkLeft = [pygame.image.load("resources/L1E.png"), pygame.image.load("resources/L2E.png"), pygame.image.load("resources/L3E.png"), pygame.image.load("resources/L4E.png"), pygame.image.load("resources/L5E.png"), pygame.image.load("resources/L6E.png"), pygame.image.load("resources/L7E.png"), pygame.image.load("resources/L8E.png"), pygame.image.load("resources/L9E.png"), pygame.image.load("resources/L10E.png"), pygame.image.load("resources/L11E.png")]
    
    def __init__(self,x,y,h,w,end):
        self.x = x
        self.y = y
        self.h = h
        self.w = w
        self.end = end
        self.walkCount = 0
        self.vel = 7
        self.path = [self.x,self.end]
        
    def draw(self,win):
        self.move()
        if self.walkCount+1 >= 33:
            self.walkCount = 0
            
        if self.vel > 0:
            win.blit(self.walkRight[self.walkCount // 3], (self.x, self.y))
            self.walkCount += 1
        else:
            win.blit(self.walkLeft[self.walkCount // 3], (self.x, self.y))
            self.walkCount += 1
    
    def move(self):
        if self.vel > 0:
            if self.x + self.vel < self.path[1]:
                self.x += self.vel
            else:
                self.vel *= -1
                self.walkCount = 0
        else:
            if self.x - self.vel > self.path[0]:
                self.x += self.vel
            else:
                self.vel *= -1
                self.walkCount = 0
    

    
man = Player(50,400,54,64)
goblin = Enemy(80,405,64,64,450)

def reDrawGameWindow():
    win.blit(bg, (0,0))
    man.draw(win)
    goblin.draw(win)
    for bullet in bullets:
        bullet.draw(win)
        
    pygame.display.update()   
     
        

bullets = []
run = True
while run:
    clock.tick(27)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
    
    for bullet in bullets:
        if bullet.x < 500 and bullet.x > 0:
            bullet.x += bullet.vel
        else:
            bullets.remove(bullet)
            
    
    keys = pygame.key.get_pressed()
    if keys[pygame.K_SPACE]:
        if man.left:
            facing = -1
        else:
            facing = 1

        if len(bullets) < 5:
            bullets.append(Projectile(round(man.x + man.w//2), round(man.y + man.h//2), 3, (0,0,0), facing))

    if keys[pygame.K_LEFT] and (man.x-man.vel) > 0:
        man.x -= man.vel
        man.left = True 
        man.right = False
        man.standing = False

    elif keys[pygame.K_RIGHT] and (man.x+man.h+man.vel) < screen_width:
        man.x += man.vel
        man.left = False 
        man.right = True
        man.standing = False

    else:
        man.standing = True
        man.walkCount = 0

    if not man.isJump:
        if keys[pygame.K_UP]:
            man.isJump = True
            man.walkCount = 0
    else:
        if man.jumpCount >= -10:
            neg = 1
            if man.jumpCount < 0:
                neg = -1
            man.y -= (man.jumpCount ** 2) *0.5 * neg
            man.jumpCount -= 1
        else:
            man.isJump = False 
            man.jumpCount = 10

    reDrawGameWindow()
            
pygame.quit()

In [16]:
# In this module we will work on collisions,
# basically collisions between the bullets and the goblin

# to do collisions we typically use boxes in pygame, because it is lot easier
# this hit boxes work like, as soon as they come together one inside another we will say that they are colliding.


# first we need to manually define the hit boxes
import pygame
pygame.init()

win = pygame.display.set_mode((500,480))
pygame.display.set_caption("FIRST GAME")
screen_width = 480
bg = pygame.image.load("resources/bg.jpg")
clock = pygame.time.Clock()


class Player():
    walkRight = [pygame.image.load("resources/R1.png"), pygame.image.load("resources/R2.png"), pygame.image.load("resources/R3.png"), pygame.image.load("resources/R4.png"), pygame.image.load("resources/R5.png"), pygame.image.load("resources/R6.png"), pygame.image.load("resources/R7.png"), pygame.image.load("resources/R8.png"), pygame.image.load("resources/R9.png")] 
    walkLeft = [pygame.image.load("resources/L1.png"), pygame.image.load("resources/L2.png"), pygame.image.load("resources/L3.png"), pygame.image.load("resources/L4.png"), pygame.image.load("resources/L5.png"), pygame.image.load("resources/L6.png"), pygame.image.load("resources/L7.png"), pygame.image.load("resources/L8.png"), pygame.image.load("resources/L9.png")]

    def __init__(self,x,y,w,h):
        self.x = x
        self.y = y
        self.h = h
        self.w = w
        self.vel = 10
        self.isJump = False
        self.jumpCount = 10
        self.left = False
        self.right = False
        self.walkCount = 0
        self.standing = True
        # creating hitbox attribute
        self.hitbox = (self.x+20, self.y+8, 28, 60) # this is a rectangle
        # the variables inside the brackets are x,y,w,h
    

    def draw(self,win):
        if self.walkCount+1 >= 27:
            self.walkCount = 0
        
        if not self.standing:
            if self.left:
                win.blit(self.walkLeft[self.walkCount//3], (self.x,self.y))
                self.walkCount += 1
            elif self.right:
                win.blit(self.walkRight[self.walkCount//3], (self.x,self.y))
                self.walkCount += 1
        else:
            if self.right:
                win.blit(self.walkRight[0], (self.x,self.y))
            else:
                win.blit(self.walkLeft[0], (self.x,self.y))
        
        # our player is moving so we need to continuously change the hitbox every loop in draw method.
        self.hitbox = (self.x+20, self.y+8, 28, 60)
        # drawing our hitbox
        pygame.draw.rect(win,(255,10,24),self.hitbox,2)
            

class Projectile():
    def __init__(self,x,y,radius,color,facing):
        self.x = x
        self.y = y
        self.radius = radius
        self.color = color
        self.facing = facing
        self.vel = 15 * facing
        
    def draw(self,win):
        pygame.draw.circle(win,self.color,(self.x,self.y),self.radius)


# creating the hitbox for the enemy
class Enemy:
    walkRight = [pygame.image.load("resources/R1E.png"), pygame.image.load("resources/R2E.png"), pygame.image.load("resources/R3E.png"), pygame.image.load("resources/R4E.png"), pygame.image.load("resources/R5E.png"), pygame.image.load("resources/R6E.png"), pygame.image.load("resources/R7E.png"), pygame.image.load("resources/R8E.png"), pygame.image.load("resources/R9E.png"), pygame.image.load("resources/R10E.png"), pygame.image.load("resources/R11E.png")] 
    walkLeft = [pygame.image.load("resources/L1E.png"), pygame.image.load("resources/L2E.png"), pygame.image.load("resources/L3E.png"), pygame.image.load("resources/L4E.png"), pygame.image.load("resources/L5E.png"), pygame.image.load("resources/L6E.png"), pygame.image.load("resources/L7E.png"), pygame.image.load("resources/L8E.png"), pygame.image.load("resources/L9E.png"), pygame.image.load("resources/L10E.png"), pygame.image.load("resources/L11E.png")]
    
    def __init__(self,x,y,w,h,end):
        self.x = x
        self.y = y
        self.h = h
        self.w = w
        self.end = end
        self.walkCount = 0
        self.vel = 7
        self.path = [self.x,self.end]
        self.hitbox = (self.x+13, self.y, 40, 60)
        
    def draw(self,win):
        self.move()
        if self.walkCount+1 >= 33:
            self.walkCount = 0
            
        if self.vel > 0:
            win.blit(self.walkRight[self.walkCount // 3], (self.x, self.y))
            self.walkCount += 1
        else:
            win.blit(self.walkLeft[self.walkCount // 3], (self.x, self.y))
            self.walkCount += 1
            
        self.hitbox = (self.x+13, self.y, 40, 60)
        pygame.draw.rect(win,(255,10,24),self.hitbox,2)
    
    def move(self):
        if self.vel > 0:
            if self.x + self.vel < self.path[1]:
                self.x += self.vel
            else:
                self.vel *= -1
                self.walkCount = 0
        else:
            if self.x - self.vel > self.path[0]:
                self.x += self.vel
            else:
                self.vel *= -1
                self.walkCount = 0
    

    
man = Player(50,400,54,64)
goblin = Enemy(80,405,64,64,450)

def reDrawGameWindow():
    win.blit(bg, (0,0))
    man.draw(win)
    goblin.draw(win)
    for bullet in bullets:
        bullet.draw(win)
        
    pygame.display.update()   
     
        

bullets = []
run = True
while run:
    clock.tick(27)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
    
    for bullet in bullets:
        if bullet.x < 500 and bullet.x > 0:
            bullet.x += bullet.vel
        else:
            bullets.remove(bullet)
            
    
    keys = pygame.key.get_pressed()
    if keys[pygame.K_SPACE]:
        if man.left:
            facing = -1
        else:
            facing = 1

        if len(bullets) < 5:
            bullets.append(Projectile(round(man.x + man.w//2), round(man.y + man.h//2), 3, (0,0,0), facing))

    if keys[pygame.K_LEFT] and (man.x-man.vel) > 0:
        man.x -= man.vel
        man.left = True 
        man.right = False
        man.standing = False

    elif keys[pygame.K_RIGHT] and (man.x+man.h+man.vel) < screen_width:
        man.x += man.vel
        man.left = False 
        man.right = True
        man.standing = False

    else:
        man.standing = True
        man.walkCount = 0

    if not man.isJump:
        if keys[pygame.K_UP]:
            man.isJump = True
            man.walkCount = 0
    else:
        if man.jumpCount >= -10:
            neg = 1
            if man.jumpCount < 0:
                neg = -1
            man.y -= (man.jumpCount ** 2) *0.5 * neg
            man.jumpCount -= 1
        else:
            man.isJump = False 
            man.jumpCount = 10

    reDrawGameWindow()
            
pygame.quit()

In [1]:
# now we have made the hitboxes, now we will work on collisions

import pygame
pygame.init()

win = pygame.display.set_mode((500,480))
pygame.display.set_caption("FIRST GAME")
screen_width = 480
bg = pygame.image.load("resources/bg.jpg")
clock = pygame.time.Clock()


class Player():
    walkRight = [pygame.image.load("resources/R1.png"), pygame.image.load("resources/R2.png"), pygame.image.load("resources/R3.png"), pygame.image.load("resources/R4.png"), pygame.image.load("resources/R5.png"), pygame.image.load("resources/R6.png"), pygame.image.load("resources/R7.png"), pygame.image.load("resources/R8.png"), pygame.image.load("resources/R9.png")] 
    walkLeft = [pygame.image.load("resources/L1.png"), pygame.image.load("resources/L2.png"), pygame.image.load("resources/L3.png"), pygame.image.load("resources/L4.png"), pygame.image.load("resources/L5.png"), pygame.image.load("resources/L6.png"), pygame.image.load("resources/L7.png"), pygame.image.load("resources/L8.png"), pygame.image.load("resources/L9.png")]

    def __init__(self,x,y,w,h):
        self.x = x
        self.y = y
        self.h = h
        self.w = w
        self.vel = 10
        self.isJump = False
        self.jumpCount = 10
        self.left = False
        self.right = False
        self.walkCount = 0
        self.standing = True
        self.hitbox = (self.x+20, self.y+8, 28, 60)    

    def draw(self,win):
        if self.walkCount+1 >= 27:
            self.walkCount = 0
        
        if not self.standing:
            if self.left:
                win.blit(self.walkLeft[self.walkCount//3], (self.x,self.y))
                self.walkCount += 1
            elif self.right:
                win.blit(self.walkRight[self.walkCount//3], (self.x,self.y))
                self.walkCount += 1
        else:
            if self.right:
                win.blit(self.walkRight[0], (self.x,self.y))
            else:
                win.blit(self.walkLeft[0], (self.x,self.y))
        
        self.hitbox = (self.x+20, self.y+8, 28, 60)
        pygame.draw.rect(win,(255,10,24),self.hitbox,2)
            

class Projectile():
    def __init__(self,x,y,radius,color,facing):
        self.x = x
        self.y = y
        self.radius = radius
        self.color = color
        self.facing = facing
        self.vel = 15 * facing
        
    def draw(self,win):
        pygame.draw.circle(win,self.color,(self.x,self.y),self.radius)


class Enemy:
    walkRight = [pygame.image.load("resources/R1E.png"), pygame.image.load("resources/R2E.png"), pygame.image.load("resources/R3E.png"), pygame.image.load("resources/R4E.png"), pygame.image.load("resources/R5E.png"), pygame.image.load("resources/R6E.png"), pygame.image.load("resources/R7E.png"), pygame.image.load("resources/R8E.png"), pygame.image.load("resources/R9E.png"), pygame.image.load("resources/R10E.png"), pygame.image.load("resources/R11E.png")] 
    walkLeft = [pygame.image.load("resources/L1E.png"), pygame.image.load("resources/L2E.png"), pygame.image.load("resources/L3E.png"), pygame.image.load("resources/L4E.png"), pygame.image.load("resources/L5E.png"), pygame.image.load("resources/L6E.png"), pygame.image.load("resources/L7E.png"), pygame.image.load("resources/L8E.png"), pygame.image.load("resources/L9E.png"), pygame.image.load("resources/L10E.png"), pygame.image.load("resources/L11E.png")]
    
    def __init__(self,x,y,w,h,end):
        self.x = x
        self.y = y
        self.h = h
        self.w = w
        self.end = end
        self.walkCount = 0
        self.vel = 7
        self.path = [self.x,self.end]
        self.hitbox = (self.x+13, self.y, 40, 60)
        
    def draw(self,win):
        self.move()
        if self.walkCount+1 >= 33:
            self.walkCount = 0
            
        if self.vel > 0:
            win.blit(self.walkRight[self.walkCount // 3], (self.x, self.y))
            self.walkCount += 1
        else:
            win.blit(self.walkLeft[self.walkCount // 3], (self.x, self.y))
            self.walkCount += 1
            
        self.hitbox = (self.x+13, self.y, 40, 60)
        pygame.draw.rect(win,(255,10,24),self.hitbox,2)
    
    def move(self):
        if self.vel > 0:
            if self.x + self.vel < self.path[1]:
                self.x += self.vel
            else:
                self.vel *= -1
                self.walkCount = 0
        else:
            if self.x - self.vel > self.path[0]:
                self.x += self.vel
            else:
                self.vel *= -1
                self.walkCount = 0
                
    # In our Enemy class, we will define a new method named hit
    def hit(self):
        # we will run this hit method each time the goblin gets hit
        print("Goblin: I will not die until you hit be little bit harder!!")

    

    
man = Player(50,400,54,64)
goblin = Enemy(80,405,64,64,450)

def reDrawGameWindow():
    win.blit(bg, (0,0))
    man.draw(win)
    goblin.draw(win)
    for bullet in bullets:
        bullet.draw(win)
        
    pygame.display.update()   
     
        
# Nw we will check for collisions in our main loop
bullets = []
run = True
while run:
    clock.tick(27)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
    
    # checking
    for bullet in bullets:
        if bullet.y - bullet.radius < goblin.hitbox[1] + goblin.hitbox[3] and bullet.y + bullet.radius > goblin.hitbox[1]:
            if bullet.x + bullet.radius > goblin.hitbox[0] and bullet.x -bullet.radius < goblin.hitbox[0] + goblin.hitbox[2]:
                goblin.hit()
                # we also want to make sure that our bullet disappear
                bullets.remove(bullet)
        if bullet.x < 500 and bullet.x > 0:
            bullet.x += bullet.vel
        else:
            bullets.remove(bullet)
            
    
    keys = pygame.key.get_pressed()
    if keys[pygame.K_SPACE]:
        if man.left:
            facing = -1
        else:
            facing = 1

        if len(bullets) < 5:
            bullets.append(Projectile(round(man.x + man.w//2), round(man.y + man.h//2), 3, (0,0,0), facing))

    if keys[pygame.K_LEFT] and (man.x-man.vel) > 0:
        man.x -= man.vel
        man.left = True 
        man.right = False
        man.standing = False

    elif keys[pygame.K_RIGHT] and (man.x+man.h+man.vel) < screen_width:
        man.x += man.vel
        man.left = False 
        man.right = True
        man.standing = False

    else:
        man.standing = True
        man.walkCount = 0

    if not man.isJump:
        if keys[pygame.K_UP]:
            man.isJump = True
            man.walkCount = 0
    else:
        if man.jumpCount >= -10:
            neg = 1
            if man.jumpCount < 0:
                neg = -1
            man.y -= (man.jumpCount ** 2) *0.5 * neg
            man.jumpCount -= 1
        else:
            man.isJump = False 
            man.jumpCount = 10

    reDrawGameWindow()
            
pygame.quit()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will 

<ipython-input-1-a34cb42ae7f3>:44: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  win.blit(self.walkRight[0], (self.x,self.y))
<ipython-input-1-a34cb42ae7f3>:49: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  pygame.draw.rect(win,(255,10,24),self.hitbox,2)
<ipython-input-1-a34cb42ae7f3>:40: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  win.blit(self.walkRight[self.walkCount//3], (self.x,self.y))


Goblin: I will not die until you hit be little bit harder!!


<ipython-input-1-a34cb42ae7f3>:37: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  win.blit(self.walkLeft[self.walkCount//3], (self.x,self.y))
<ipython-input-1-a34cb42ae7f3>:46: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  win.blit(self.walkLeft[0], (self.x,self.y))


Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be 

In [8]:
# Now we can see that everything works fine, but we have a little problem
# we shoot multiple bullets at a time and we need to fix that issue

import pygame
pygame.init()

win = pygame.display.set_mode((500,480))
pygame.display.set_caption("FIRST GAME")
screen_width = 480
bg = pygame.image.load("resources/bg.jpg")
clock = pygame.time.Clock()


class Player():
    walkRight = [pygame.image.load("resources/R1.png"), pygame.image.load("resources/R2.png"), pygame.image.load("resources/R3.png"), pygame.image.load("resources/R4.png"), pygame.image.load("resources/R5.png"), pygame.image.load("resources/R6.png"), pygame.image.load("resources/R7.png"), pygame.image.load("resources/R8.png"), pygame.image.load("resources/R9.png")] 
    walkLeft = [pygame.image.load("resources/L1.png"), pygame.image.load("resources/L2.png"), pygame.image.load("resources/L3.png"), pygame.image.load("resources/L4.png"), pygame.image.load("resources/L5.png"), pygame.image.load("resources/L6.png"), pygame.image.load("resources/L7.png"), pygame.image.load("resources/L8.png"), pygame.image.load("resources/L9.png")]

    def __init__(self,x,y,w,h):
        self.x = x
        self.y = y
        self.h = h
        self.w = w
        self.vel = 10
        self.isJump = False
        self.jumpCount = 10
        self.left = False
        self.right = False
        self.walkCount = 0
        self.standing = True
        self.hitbox = (self.x+20, self.y+8, 28, 60)    

    def draw(self,win):
        if self.walkCount+1 >= 27:
            self.walkCount = 0
        
        if not self.standing:
            if self.left:
                win.blit(self.walkLeft[self.walkCount//3], (self.x,self.y))
                self.walkCount += 1
            elif self.right:
                win.blit(self.walkRight[self.walkCount//3], (self.x,self.y))
                self.walkCount += 1
        else:
            if self.right:
                win.blit(self.walkRight[0], (self.x,self.y))
            else:
                win.blit(self.walkLeft[0], (self.x,self.y))
        
        self.hitbox = (self.x+20, self.y+8, 28, 60)
        pygame.draw.rect(win,(255,10,24),self.hitbox,2)
            

class Projectile():
    def __init__(self,x,y,radius,color,facing):
        self.x = x
        self.y = y
        self.radius = radius
        self.color = color
        self.facing = facing
        self.vel = 15 * facing
        
    def draw(self,win):
        pygame.draw.circle(win,self.color,(self.x,self.y),self.radius)


class Enemy:
    walkRight = [pygame.image.load("resources/R1E.png"), pygame.image.load("resources/R2E.png"), pygame.image.load("resources/R3E.png"), pygame.image.load("resources/R4E.png"), pygame.image.load("resources/R5E.png"), pygame.image.load("resources/R6E.png"), pygame.image.load("resources/R7E.png"), pygame.image.load("resources/R8E.png"), pygame.image.load("resources/R9E.png"), pygame.image.load("resources/R10E.png"), pygame.image.load("resources/R11E.png")] 
    walkLeft = [pygame.image.load("resources/L1E.png"), pygame.image.load("resources/L2E.png"), pygame.image.load("resources/L3E.png"), pygame.image.load("resources/L4E.png"), pygame.image.load("resources/L5E.png"), pygame.image.load("resources/L6E.png"), pygame.image.load("resources/L7E.png"), pygame.image.load("resources/L8E.png"), pygame.image.load("resources/L9E.png"), pygame.image.load("resources/L10E.png"), pygame.image.load("resources/L11E.png")]
    
    def __init__(self,x,y,w,h,end):
        self.x = x
        self.y = y
        self.h = h
        self.w = w
        self.end = end
        self.walkCount = 0
        self.vel = 7
        self.path = [self.x,self.end]
        self.hitbox = (self.x+13, self.y, 40, 60)
        
    def draw(self,win):
        self.move()
        if self.walkCount+1 >= 33:
            self.walkCount = 0
            
        if self.vel > 0:
            win.blit(self.walkRight[self.walkCount // 3], (self.x, self.y))
            self.walkCount += 1
        else:
            win.blit(self.walkLeft[self.walkCount // 3], (self.x, self.y))
            self.walkCount += 1
            
        self.hitbox = (self.x+13, self.y, 40, 60)
        pygame.draw.rect(win,(255,10,24),self.hitbox,2)
    
    def move(self):
        if self.vel > 0:
            if self.x + self.vel < self.path[1]:
                self.x += self.vel
            else:
                self.vel *= -1
                self.walkCount = 0
        else:
            if self.x - self.vel > self.path[0]:
                self.x += self.vel
            else:
                self.vel *= -1
                self.walkCount = 0
                
    def hit(self):
        print("Goblin: I will not die until you hit be little bit harder!!")

    

    
man = Player(50,400,54,64)
goblin = Enemy(80,405,64,64,450)

# creating shootLoop variable
shootLoop =0

def reDrawGameWindow():
    win.blit(bg, (0,0))
    man.draw(win)
    goblin.draw(win)
    for bullet in bullets:
        bullet.draw(win)
        
    pygame.display.update()   
     
        
bullets = []
run = True
while run:
    clock.tick(27)
    
    # changing the main loop
    # it will basically set a timer so that we cant shoot multiple bullets at a time.
    if shootLoop > 0:
        shootLoop += 1
    if shootLoop > 3:
        shootLoop = 0
        
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
    
    for bullet in bullets:
        if bullet.y - bullet.radius < goblin.hitbox[1] + goblin.hitbox[3] and bullet.y + bullet.radius > goblin.hitbox[1]:
            if bullet.x + bullet.radius > goblin.hitbox[0] and bullet.x -bullet.radius < goblin.hitbox[0] + goblin.hitbox[2]:
                goblin.hit()
                # we also want to make sure that our bullet disappear
                bullets.remove(bullet)
        if bullet.x < 500 and bullet.x > 0:
            bullet.x += bullet.vel
        else:
            bullets.remove(bullet)
            
    
    keys = pygame.key.get_pressed()
    
    # changing the bullet shooting condition
    if keys[pygame.K_SPACE] and shootLoop == 0:
        if man.left:
            facing = -1
        else:
            facing = 1

        if len(bullets) < 5:
            bullets.append(Projectile(round(man.x + man.w//2), round(man.y + man.h//2), 3, (0,0,0), facing))
            
        # changing the shootloop variable
        shootLoop = 1

    if keys[pygame.K_LEFT] and (man.x-man.vel) > 0:
        man.x -= man.vel
        man.left = True 
        man.right = False
        man.standing = False

    elif keys[pygame.K_RIGHT] and (man.x+man.h+man.vel) < screen_width:
        man.x += man.vel
        man.left = False 
        man.right = True
        man.standing = False

    else:
        man.standing = True
        man.walkCount = 0

    if not man.isJump:
        if keys[pygame.K_UP]:
            man.isJump = True
            man.walkCount = 0
    else:
        if man.jumpCount >= -10:
            neg = 1
            if man.jumpCount < 0:
                neg = -1
            man.y -= (man.jumpCount ** 2) *0.5 * neg
            man.jumpCount -= 1
        else:
            man.isJump = False 
            man.jumpCount = 10

    reDrawGameWindow()
            
pygame.quit()

Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!


<ipython-input-8-10fee17f5aa9>:41: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  win.blit(self.walkRight[self.walkCount//3], (self.x,self.y))
<ipython-input-8-10fee17f5aa9>:50: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  pygame.draw.rect(win,(255,10,24),self.hitbox,2)
<ipython-input-8-10fee17f5aa9>:45: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  win.blit(self.walkRight[0], (self.x,self.y))


Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!


<ipython-input-8-10fee17f5aa9>:38: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  win.blit(self.walkLeft[self.walkCount//3], (self.x,self.y))
<ipython-input-8-10fee17f5aa9>:47: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  win.blit(self.walkLeft[0], (self.x,self.y))


Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
Goblin: I will not die until you hit be little bit harder!!
